## 집 값 예측
- 예측할 변수 ['SalePrice']
- 평가: rmse, r2

    - rmse는 낮을 수록 좋은 성능
    - r2는 높을 수록 좋은 성능
   

In [1]:
# import

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# RMSE 함수 정의 (회귀의 핵심 평가지표)
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [2]:
# 1.데이터 로드
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print('-----------------------------')
print('데이터 정보 확인')
print(train.info())
print('-----------------------------')
print('데이터 예시 확인')
print(train.head())
print('-----------------------------')
print("결측치 확인:")
print(train.isnull().sum())

Train shape: (1460, 81)
Test shape: (1459, 80)
-----------------------------
데이터 정보 확인
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyl

In [3]:
# 2. 타켓 분리
# SalePrice 예측하는 회귀 문제

# 현재 예측하고자 하는 것이 SalePrice 이를 제거한 데이터셋을 만든다.
X_train = train.drop('SalePrice', axis=1)  # 타겟 제거
y_train = train['SalePrice']

print('y_train예시')
print(y_train.head(10))
print('------------------------')
print('X_train예시')
print(X_train.head(10))
print('------------------------')
print('타겟 분포')
print(y_train.describe())

y_train예시
0    208500
1    181500
2    223500
3    140000
4    250000
5    143000
6    307000
7    200000
8    129900
9    118000
Name: SalePrice, dtype: int64
------------------------
X_train예시
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN    

In [4]:
# 3. 결측치 처리
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
print(f'numeric_cols:{numeric_cols}')

for col in numeric_cols:
    median_val = X_train[col].median()
    
    # X_train이랑 test에 같은 값을 대입한다.
    X_train[col] = X_train[col].fillna(median_val)
    test[col] = test[col].fillna(median_val)

# 범주형: 최빈값으로 처리
categorical_cols = X_train.select_dtypes(include=['object']).columns
print(f'categorical_cols:{categorical_cols}')

for col in categorical_cols:
    mode_val = X_train[col].mode()[0]
    
    # X_train이랑 test에 같은 값을 대입한다.
    X_train[col] = X_train[col].fillna(mode_val)
    test[col] = test[col].fillna(mode_val)

print("결측치 확인:")
print(X_train.isnull().sum())
print(test.isnull().sum())

numeric_cols:Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')
categorical_cols:Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
  

In [5]:
# 4. 인코딩
# 범주형 변수 레이블 인코딩
le_dict = {}  # 인코더 저장소
for col in categorical_cols:
    le = LabelEncoder()
    # train + test 합쳐서 fit (중요!)
    all_values = pd.concat([X_train[col], test[col]], axis=0)
    # 합친 것을 학습
    le.fit(all_values)
    
    # 학습 후, X_train,test에 변환
    X_train[col] = le.transform(X_train[col])
    test[col] = le.transform(test[col])
    
    # 학습된 값을 le_dict에 저장. 나중에 새로운 데이터가 와도 같은 매핑을 사용하도록
    le_dict[col] = le

# 잘 되었나 확인하기
print(f'X_train:{X_train.head(10)}')
print('-----------------------')
print(f'test:{test.head(10)}')

X_train:   Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  \
0   1          60         3         65.0     8450       1      0         3   
1   2          20         3         80.0     9600       1      0         3   
2   3          60         3         68.0    11250       1      0         0   
3   4          70         3         60.0     9550       1      0         0   
4   5          60         3         84.0    14260       1      0         0   
5   6          50         3         85.0    14115       1      0         0   
6   7          20         3         75.0    10084       1      0         3   
7   8          60         3         69.0    10382       1      0         0   
8   9          50         4         51.0     6120       1      0         3   
9  10         190         3         50.0     7420       1      0         3   

   LandContour  Utilities  ...  ScreenPorch  PoolArea  PoolQC  Fence  \
0            3          0  ...            0         0       2

In [6]:
# 5. 검증 데이터 분할
# 검증 데이터를 통해 미리 성능 확인!
# 80% 학습용, 20% 검증용

# 회귀문제=stratify 사용하지 않음!(연속형 타겟이므로)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print('X_train 데이터 분리 확인')
print(X_train.shape)
print(X_tr.shape)
print(X_val.shape)

print('y_train 데이터 분리 확인')
print(y_train.shape)
print(y_tr.shape)
print(y_val.shape)

X_train 데이터 분리 확인
(1460, 80)
(1168, 80)
(292, 80)
y_train 데이터 분리 확인
(1460,)
(1168,)
(292,)


In [7]:
# 6. 모델 학습 및 평가
# 모델 선택
model = RandomForestRegressor(random_state=42, n_estimators=200)

# 모델 학습
model.fit(X_tr, y_tr)

# 검증 성능 확인
# 회귀에서는 predict 사용 (predict_proba 아님)
val_pred = model.predict(X_val)

# 회귀 평가지표들 (낮을수록 좋음: MSE, MAE, RMSE / 높을수록 좋음: R²)
print(f"MSE: {mean_squared_error(y_val, val_pred)}")
print(f"MAE: {mean_absolute_error(y_val, val_pred)}")
print(f"RMSE: {rmse(y_val, val_pred)}")
print(f"RMSE/MAE: {rmse(y_val, val_pred)/mean_absolute_error(y_val, val_pred)}")
print(f"R²: {r2_score(y_val, val_pred)}")

MSE: 827086075.4322888
MAE: 17627.830291095892
RMSE: 28759.10421818261
RMSE/MAE: 1.6314602389103614
R²: 0.8921706778706092


In [8]:
# 7. 최종 예측 및 제출
# 전체 데이터로 학습
# 시간이 있다면 파라미터 조정(max_depth=10)
final_model = RandomForestRegressor(random_state=42, n_estimators=200)

# 모델 학습
final_model.fit(X_train, y_train)

# 회귀에서는 predict 사용
test_pred = final_model.predict(test)

# 제출 파일 생성
submit = pd.DataFrame({'pred': test_pred})
submit.to_csv('submission_house.csv', index=False)
print("제출 파일 생성 완료!")
print(submit.head(10))

제출 파일 생성 완료!
         pred
0  128151.830
1  155486.160
2  180680.955
3  180180.035
4  198038.765
5  184002.820
6  165673.045
7  175259.110
8  181840.880
9  122229.915
